# Exporting Diaz2019 Coordinates

Code for exporting xy coordinates for all figures in the paper

### Import Packages

In [1]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc
import scipy.sparse
import scrublet as scr
import os
import sys
import pickle

sys.path.append('../')
import helper_functions_dew as dew

In [2]:
# ScanPy settings
sc.settings.verbosity = 0  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=120, dpi_save=600, vector_friendly=False)  
sc.logging.print_versions()

# Matplotlib settings
plt.rcParams['pdf.fonttype']=42  # Necessary to export text (rather than curves) in pdf files

scanpy==0+unknown anndata==0.6.19 umap==0.3.8 numpy==1.16.2 scipy==1.2.1 pandas==0.24.2 scikit-learn==0.20.3 statsmodels==0.9.0 python-igraph==0.7.1 louvain==0.6.1 


### Load Fully Analyzed DataSets

In [3]:
# Mouse E95
mmE95 = sc.read('../mmE95/Diaz2019_mmE95.h5ad')
print(mmE95); print()

# Mouse E95 Doublet-filtered
mmE95_df = sc.read('../mmE95/Diaz2019_mmE95df.h5ad')
print(mmE95_df); print()

# Mouse E95 Neural Tube + PSM
mmE95_df_sub = sc.read('../mmE95/Diaz2019_mmE95dfsub.h5ad')
print(mmE95_df_sub); print()

# Mouse E95 PSM only
mmE95_df_subsub = sc.read('../mmE95/Diaz2019_mmE95dfsubsub.h5ad')
print(mmE95_df_subsub); print()

# Mouse ESC
mmESC = sc.read('../mmES/Diaz2019_mmES_analyzed.h5ad')
print(mmESC); print()

# Human iPSC
hsIPSC = sc.read('../hsIPS/Diaz2019_hsiPS_full.h5ad')
print(hsIPSC); print()

# Load E95 Classifier object
E95Classifier = pickle.load(open('../mmE95/E95Classifier.pickle','rb'))

AnnData object with n_obs × n_vars = 5646 × 40523 
    obs: 'batch', 'cell_names', 'library_id', 'n_counts', 'unique_cell_id'

AnnData object with n_obs × n_vars = 4367 × 40523 
    obs: 'batch', 'cell_names', 'library_id', 'n_counts', 'unique_cell_id', 'doublet_scores', 'predicted_doublets', 'n_genes', 'n_counts_pre_norm', 'leiden', 'louvain'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'leiden', 'leiden_colors', 'louvain', 'louvain_colors', 'neighbors', 'pca'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    varm: 'PCs'

AnnData object with n_obs × n_vars = 2340 × 40523 
    obs: 'batch', 'cell_names', 'library_id', 'n_counts', 'unique_cell_id', 'doublet_scores', 'predicted_doublets', 'n_genes', 'n_counts_pre_norm', 'leiden', 'louvain', 'dpt_pseudotime'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'diffmap_evals', 'draw_graph', 'iroot', 'leiden', 'leiden_colors', 'library_id_colors', 'louvain', 'louvain_colors', 'louvain_s

### Export Coordinates Tables

In some cases, we need to re-execute code from the original notebooks to recover plotting coordinates.  Rather than modifying those notebooks, just duplicate the relevant code here.

#### Figure 2

In [ ]:
# Fig2A: mmE95_df_sub FA + Louvain
df = pd.DataFrame(mmE95_df_sub.obsm['X_draw_graph_fa'], index=mmE95_df_sub.obs['louvain'], columns=['FA_1','FA_2']) 
df.to_csv('Fig2A.csv')

# Fig2B-1: mmE95_df_sub FA + dpt pseudotime
arr = np.vstack([mmE95_df_sub.obsm['X_draw_graph_fa'][:,0], mmE95_df_sub.obsm['X_draw_graph_fa'][:,1], mmE95_df_sub.obs['dpt_pseudotime']])
df = pd.DataFrame(np.transpose(arr), columns=['FA_1','FA_2','dpt_pseudotime'], index=mmE95_df_sub.obs['louvain']) 
df.to_csv('Fig2B-1.csv')

# Fig2B-2: mmE95_df_subsub all dynamic genes PSM trajectory heatmap
dyn_genes = mmE95_df_subsub.var[mmE95_df_subsub.var['dyn_fdr_flag']].sort_values(by=['dyn_peak_cell'])
dyn_genes['dyn_gene_ord']=[ind for ind in range(1,len(dyn_genes)+1)]
traj_nodes = ['NMP','MPC','pPSM','aPSM','dmSOM']
ax,df = sc.pl.paga_path(mmE95_df_sub, nodes=traj_nodes, keys=dyn_genes.index, use_raw=True, annotations=['dpt_pseudotime'],
                     color_map='OrRd', color_maps_annotations={'dpt_pseudotime': 'jet'}, palette_groups=None, 
                     n_avg=100, groups_key=None, xlim=[None, None], title=None, left_margin=None, ytick_fontsize=1, 
                     title_fontsize=None, show_node_names=False, show_yticks=True, show_colorbar=True, 
                     legend_fontsize=1, legend_fontweight=None, normalize_to_zero_one=True, as_heatmap=True, 
                     return_data=True, show=False, ax=None)
df.to_csv('Fig2B-2.csv')

# Fig2B-3: mmE95_df_subsub select marker gene PSM trajectory heatmap
traj_nodes = ['NMP','MPC','pPSM','aPSM','dmSOM']
traj_genes = ['Sox2','Fgf17','T','Mixl1','Wnt3a','Rspo3','Msgn1','Tbx6','Dll3','Foxc1','Mesp1','Ripply2','Pax3','Mest']
ax,df = sc.pl.paga_path(mmE95_df_sub, nodes=traj_nodes, keys=traj_genes, use_raw=True, annotations=['dpt_pseudotime'],
                     color_map='OrRd', color_maps_annotations={'dpt_pseudotime': 'jet'}, palette_groups=None, 
                     n_avg=100, groups_key=None, xlim=[None, None], title=None, left_margin=None, ytick_fontsize=6, 
                     title_fontsize=None, show_node_names=False, show_yticks=True, show_colorbar=True, 
                     legend_fontsize=1, legend_fontweight=None, normalize_to_zero_one=True, as_heatmap=True, 
                     return_data=True, show=False, ax=None)
df.to_csv('Fig2B-3.csv')

# Fig2C: mmESC + Louvain
df = pd.DataFrame(mmESC.obsm['X_draw_graph_fa'], index=mmESC.obs['louvain'], columns=['FA_1','FA_2']) 
df.to_csv('Fig2C.csv')

# Fig2D: mmESC select marker gene PSM trajectory heatmap
traj_genes = traj_genes = ['Sox2','Nodal','T','Mixl1','Fgf17','Msgn1','Wnt3a','Rspo3','Tbx6','Dll3','Foxc1','Hes7','Lfng']
ax,df = sc.pl.paga_path(mmESC, nodes=['PSM'], keys=traj_genes, use_raw=True, annotations=['dpt_pseudotime'],
                     color_map='OrRd', color_maps_annotations={'dpt_pseudotime': 'jet'}, palette_groups=None, 
                     n_avg=100, groups_key='PAGAFlag', xlim=[None, None], title=None, left_margin=None, ytick_fontsize=6, 
                     title_fontsize=None, show_node_names=False, show_yticks=True, show_colorbar=True, 
                     legend_fontsize=1, legend_fontweight=None, normalize_to_zero_one=True, as_heatmap=True, 
                     return_data=True, show=False, ax=None)
df.to_csv('Fig2D.csv')

# Fig2E: hsIPSC + Louvain
df = pd.DataFrame(hsIPSC.obsm['X_draw_graph_fa'], index=hsIPSC.obs['louvain'], columns=['FA_1','FA_2']) 
df.to_csv('Fig2E.csv')

# Fig2F: hIPSC select marker gene PSM trajectory heatmap
traj_genes = ['SOX2','NODAL','T','MIXL1','FGF17','MSGN1','WNT3A','RSPO3','TBX6','DLL3','FOXC1','LFNG','HES7']
ax,df = sc.pl.paga_path(hsIPSC, nodes=['PSM'], keys=traj_genes, use_raw=True, annotations=['dpt_pseudotime'],
                     color_map='OrRd', color_maps_annotations={'dpt_pseudotime': 'jet'}, palette_groups=None, 
                     n_avg=100, groups_key='PAGAFlag', xlim=[None, None], title=None, left_margin=None, ytick_fontsize=6, 
                     title_fontsize=None, show_node_names=False, show_yticks=True, show_colorbar=True, 
                     legend_fontsize=1, legend_fontweight=None, normalize_to_zero_one=True, as_heatmap=True, 
                     return_data=True, show=False, ax=None)
df.to_csv('Fig2F.csv')

# Fig2G: mmE95 classifier scores for mmESC and hsIPSC presumptive pPSM cell clusters
hsiPS_PSM = hsIPSC[hsIPSC.obs['louvain'].isin(['d2 MPC','d3-4 aPSM']),:]
hsiPS_PSM.obs['all']='d2-4 MPC-aPSM'
fig,ax,hsiPS_scores,hsiPS_x_names,hsiPS_y_names = dew.plot_confusion_matrix(hsiPS_PSM.obs['pr_NearestNeighbors'],hsiPS_PSM.obs['all'], title=' ', normalize=True, cmap='viridis', vmin=0, vmax=0.3, return_data=True)
mmES_pPSM = mmESC[mmESC.obs['louvain'].isin(['d4-5 pPSM']),:]
fig,ax,mmES_scores,mmES_x_names,mmES_y_names = dew.plot_confusion_matrix(mmES_pPSM.obs['pr_NearestNeighbors'],mmES_pPSM.obs['louvain'], title=' ', normalize=True, cmap='viridis', vmin=0, vmax=0.3, return_data=True)
arr = np.vstack([mmES_scores, hsiPS_scores])
df = pd.DataFrame(np.transpose(arr), columns=['mESC_pPSM_Score','hsIPSC_pPSM_Score'], index=mmES_x_names) 
df.to_csv('Fig2G.csv')

# Fig2H-1: mmESC + E95PSM prediction
score = mmESC.obsm['proba_NearestNeighbors'][:,np.where(E95Classifier['Classes']=='pPSM')[0][0]]
arr = np.vstack([mmESC.obsm['X_draw_graph_fa'][:,0],[mmESC.obsm['X_draw_graph_fa'][:,1], score]])
df = pd.DataFrame(np.transpose(arr), columns=['FA_1','FA_2','pPSM_score']) 
df.to_csv('Fig2H-1.csv')

# Fig2H-2: hsIPSC + E95PSM prediction
score = hsIPSC.obsm['proba_NearestNeighbors'][:,np.where(E95Classifier['Classes']=='pPSM')[0][0]]
arr = np.vstack([hsIPSC.obsm['X_draw_graph_fa'][:,0],[hsIPSC.obsm['X_draw_graph_fa'][:,1], score]])
df = pd.DataFrame(np.transpose(arr), columns=['FA_1','FA_2','pPSM_score']) 
df.to_csv('Fig2H-2.csv')

#### Extended Figure 2

In [ ]:
# Fig S2A: pre-doublet filtered mmE95 UMAP with scrublet score overlay
import scrublet as scr
np.random.seed(802) # set random seed for reproducibility
scrub = scr.Scrublet(mmE95.X, n_neighbors = 20, expected_doublet_rate=0.25)
mmE95.obs['doublet_scores'], mmE95.obs['predicted_doublets'] = scrub.scrub_doublets()
scrublet_umap = scr.get_umap(scrub.manifold_obs_, n_neighbors=10, min_dist=0.2)
score = mmE95.obs['doublet_scores']
arr = np.vstack([scrublet_umap[:,0], scrublet_umap[:,1], score])
df = pd.DataFrame(np.transpose(arr), columns=['UMAP_1','UMAP_2','Scrublet_score']) 
df.to_csv('FigS2A.csv')

# Fig S2B: scrublet score histogram
arr = np.array([score])
df = pd.DataFrame(np.transpose(arr), columns=['Scrublet_score']) 
df.to_csv('FigS2B.csv')

# Fig S2C: mmE95_df tSNE + Louvain
df = pd.DataFrame(mmE95_df.obsm['X_tsne'], index=mmE95_df.obs['louvain'], columns=['tSNE_1','tSNE_2']) 
df.to_csv('FigS2C.csv')

#### Extended Figure 3

In [ ]:
# Fig S3A-1: mmE95_df_sub FA + Louvain
df = pd.DataFrame(mmE95_df_sub.obsm['X_draw_graph_fa'], index=mmE95_df_sub.obs['louvain'], columns=['FA_1','FA_2']) 
df.to_csv('FigS3A-1.csv')

# Fig S3A-2: mmE95_df_sub FA + Leiden
df = pd.DataFrame(mmE95_df_sub.obsm['X_draw_graph_fa'], index=mmE95_df_sub.obs['leiden'], columns=['FA_1','FA_2']) 
df.to_csv('FigS3A-2.csv')

# Fig S3B: Confusion mmE95 louvain vs leiden
fig,ax,scores,x_names,y_names = dew.plot_confusion_matrix(mmE95_df_sub.obs['leiden'], mmE95_df_sub.obs['louvain'], normalize=True, overlay_values=True, return_data=True)
df = pd.DataFrame(np.transpose(scores), columns=y_names, index=x_names) 
df.to_csv('FigS3B.csv')

# Fig S3D: kNN overlays
arr = np.vstack([mmE95_df_sub.obsm['X_draw_graph_fa'][:,0], mmE95_df_sub.obsm['X_draw_graph_fa'][:,1], 
                 mmE95_df_sub[:,'Sox2'].X, 
                 mmE95_df_sub[:,'Rspo3'].X, 
                 mmE95_df_sub[:,'Fgf5'].X, 
                 mmE95_df_sub[:,'T'].X, 
                 mmE95_df_sub[:,'Foxc1'].X,
                 mmE95_df_sub[:,'Nodal'].X, 
                 mmE95_df_sub[:,'Hes7'].X, 
                 mmE95_df_sub[:,'Lfng'].X])
df = pd.DataFrame(np.transpose(arr), columns=['FA_1','FA_2','Sox2', 'Rspo3','Fgf5','T','Foxc1','Nodal','Hes7','Lfng']) 
df.to_csv('FigS3D.csv')

# Fig S3E: Heatmaps
mouse_markers=['Nanog','Pou5f1','Sox2','Cdh1','Epcam','Dusp6','Spry2','Spry4','Nodal','Fgf5','Mixl1','T','Cdh2','Cdx2','Cdx1','Wnt3a','Dkk1','Axin2','Rspo3','Tbx6','Fgf17','Snai2','Dll1','Dll3','Hes7','Lfng','Heyl','Hes1','Nrarp','Notch1','Foxc1','Aldh1a2','Meis2']
traj_nodes = ['NMP','MPC','pPSM','aPSM','dmSOM']
ax,df = sc.pl.paga_path(mmE95_df_sub, nodes=traj_nodes, keys=mouse_markers, use_raw=True, annotations=['dpt_pseudotime'],
                     color_map='Reds', color_maps_annotations={'dpt_pseudotime': 'jet'}, palette_groups=None, 
                     n_avg=10, groups_key=None, xlim=[None, None], title=None, left_margin=None, ytick_fontsize=6, 
                     title_fontsize=None, show_node_names=False, show_yticks=True, show_colorbar=True, 
                     legend_fontsize=1, legend_fontweight=None, normalize_to_zero_one=True, as_heatmap=True, 
                     return_data=True, show=False, ax=None)
df.to_csv('FigS3E.csv')

# Fig S3F-1: mmESC + Louvain
df = pd.DataFrame(mmESC.obsm['X_draw_graph_fa'], index=mmESC.obs['louvain'], columns=['FA_1','FA_2']) 
df.to_csv('FigS3F-1.csv')

# Fig S3F-2: mmESC + TimeID
df = pd.DataFrame(mmESC.obsm['X_draw_graph_fa'], index=mmESC.obs['time_id'], columns=['FA_1','FA_2']) 
df.to_csv('FigS3F-2.csv')

# Fig S3G: Confusion mmESC louvain vs time
fig,ax,scores,x_names,y_names = dew.plot_confusion_matrix(mmESC.obs['time_id'], mmESC.obs['louvain'], normalize=True, overlay_values=True, return_data=True)
df = pd.DataFrame(np.transpose(scores), columns=y_names, index=x_names) 
df.to_csv('FigS3G.csv')

# Fig S3I: kNN overlays
arr = np.vstack([mmESC.obsm['X_draw_graph_fa'][:,0], mmESC.obsm['X_draw_graph_fa'][:,1], 
                 mmESC[:,'Sox2'].X, 
                 mmESC[:,'Rspo3'].X, 
                 mmESC[:,'Fgf5'].X, 
                 mmESC[:,'T'].X, 
                 mmESC[:,'Foxc1'].X,
                 mmESC[:,'Nodal'].X, 
                 mmESC[:,'Hes7'].X, 
                 mmESC[:,'Lfng'].X])
df = pd.DataFrame(np.transpose(arr), columns=['FA_1','FA_2','Sox2', 'Rspo3','Fgf5','T','Foxc1','Nodal','Hes7','Lfng']) 
df.to_csv('FigS3I.csv')

# Fig S3J: Heatmaps
mouse_markers=['Nanog','Pou5f1','Sox2','Cdh1','Epcam','Dusp6','Spry2','Spry4','Nodal','Fgf5','Mixl1','T','Cdh2','Cdx2','Cdx1','Wnt3a','Dkk1','Axin2','Rspo3','Tbx6','Fgf17','Snai2','Dll1','Dll3','Hes7','Lfng','Heyl','Hes1','Nrarp','Notch1','Foxc1','Aldh1a2','Meis2']
traj_nodes = ['PSM']
ax,df = sc.pl.paga_path(mmESC, nodes=traj_nodes, keys=mouse_markers, use_raw=True, annotations=['dpt_pseudotime'],
                     color_map='Reds', color_maps_annotations={'dpt_pseudotime': 'jet'}, palette_groups=None, 
                     n_avg=10, groups_key=None, xlim=[None, None], title=None, left_margin=None, ytick_fontsize=6, 
                     title_fontsize=None, show_node_names=False, show_yticks=True, show_colorbar=True, 
                     legend_fontsize=1, legend_fontweight=None, normalize_to_zero_one=True, as_heatmap=True, 
                     return_data=True, show=False, ax=None)
df.to_csv('FigS3J.csv')

# Fig S3K-1: hsIPSC + Louvain
df = pd.DataFrame(hsIPSC.obsm['X_draw_graph_fa'], index=hsIPSC.obs['louvain'], columns=['FA_1','FA_2']) 
df.to_csv('FigS3K-1.csv')

# Fig S3K-2: hsIPSC + TimeID
df = pd.DataFrame(hsIPSC.obsm['X_draw_graph_fa'], index=hsIPSC.obs['time_id'], columns=['FA_1','FA_2']) 
df.to_csv('FigS3K-2.csv')

# Fig S3L: Confusion hsIPSC louvain vs time
hsiPS_sub = hsIPSC[hsIPSC.obs['louvain'].isin(['d3-4 aPSM', 'd2 MPC', 'd1 NMP', 'd0 iPSC', 'str']),:]
fig,ax,scores,x_names,y_names = dew.plot_confusion_matrix(hsiPS_sub.obs['time_id'], hsiPS_sub.obs['louvain'], normalize=True, overlay_values=True, return_data=True)
df = pd.DataFrame(np.transpose(scores), columns=y_names, index=x_names) 
df.to_csv('FigS3L.csv')

# Fig S3N: kNN overlays
arr = np.vstack([hsIPSC.obsm['X_draw_graph_fa'][:,0], hsIPSC.obsm['X_draw_graph_fa'][:,1], 
                 hsIPSC[:,'SOX2'].X, 
                 hsIPSC[:,'RSPO3'].X, 
                 hsIPSC[:,'FGF5'].X, 
                 hsIPSC[:,'T'].X, 
                 hsIPSC[:,'FOXC1'].X,
                 hsIPSC[:,'NODAL'].X, 
                 hsIPSC[:,'HES7'].X, 
                 hsIPSC[:,'LFNG'].X])
df = pd.DataFrame(np.transpose(arr), columns=['FA_1','FA_2','SOX2', 'RSPO3','FGF5','T','FOXC1','NODAL','HES7','LFNG']) 
df.to_csv('FigS3N.csv')

# Fig S3O: Heatmaps
mouse_markers=['NANOG','POU5F1','SOX2','CDH1','EPCAM','DUSP6','SPRY2','SPRY4','NODAL','FGF5','MIXL1','T','CDH2','CDX2','CDX1','WNT3A','DKK1','AXIN2','RSPO3','TBX6','FGF17','SNAI2','DLL1','DKK3','HES7','LFNG','HEYL','HES1','NRARP','NOTCH1','FOXC1','ALDH1A2','MEIS2']
traj_nodes = ['PSM']
ax,df = sc.pl.paga_path(hsIPSC, nodes=traj_nodes, keys=mouse_markers, use_raw=True, annotations=['dpt_pseudotime'],
                     color_map='Reds', color_maps_annotations={'dpt_pseudotime': 'jet'}, palette_groups=None, 
                     n_avg=10, groups_key=None, xlim=[None, None], title=None, left_margin=None, ytick_fontsize=6, 
                     title_fontsize=None, show_node_names=False, show_yticks=True, show_colorbar=True, 
                     legend_fontsize=1, legend_fontweight=None, normalize_to_zero_one=True, as_heatmap=True, 
                     return_data=True, show=False, ax=None)
df.to_csv('FigS3O.csv')

#### Extended Figure 4

In [33]:
# Fig4A: mmESC + E95PSM predictions
score1 = mmESC.obsm['proba_NeuralNet'][:,np.where(E95Classifier['Classes']=='pPSM')[0][0]]
score2 = mmESC.obsm['proba_LDA'][:,np.where(E95Classifier['Classes']=='pPSM')[0][0]]
score3 = mmESC.obsm['proba_RandomForest'][:,np.where(E95Classifier['Classes']=='pPSM')[0][0]]
arr = np.vstack([mmESC.obsm['X_draw_graph_fa'][:,0],[mmESC.obsm['X_draw_graph_fa'][:,1], score1, score2, score3]])
df = pd.DataFrame(np.transpose(arr), columns=['FA_1','FA_2','NeuralNet','LDA','RandomForest']) 
df.to_csv('FigS4A.csv')

# Fig4B: mmESC + HOX
mouse_hox=['Hoxa1','Hoxb1','Hoxd1','Hoxa2','Hoxb2','Hoxa3','Hoxb3','Hoxd3','Hoxa4','Hoxb4','Hoxc4','Hoxd4','Hoxa5','Hoxb5','Hoxc5','Hoxa6','Hoxb6','Hoxc6','Hoxa7','Hoxb7','Hoxb8','Hoxc8','Hoxd8','Hoxa9','Hoxb9','Hoxc9','Hoxd9','Hoxa10','Hoxc10','Hoxd10','Hoxa11','Hoxc11','Hoxd11','Hoxc12','Hoxd12','Hoxa13','Hoxb13','Hoxc13','Hoxd13']    
arr = np.array(mmESC[:,mouse_hox].X)
df = pd.DataFrame((arr), columns=mouse_hox, index=mmESC.obs['time_id']) 
df.to_csv('FigS4B.csv')

# Fig4C: hsIPSC + E95PSM predictions
score1 = hsIPSC.obsm['proba_NeuralNet'][:,np.where(E95Classifier['Classes']=='pPSM')[0][0]]
score2 = hsIPSC.obsm['proba_LDA'][:,np.where(E95Classifier['Classes']=='pPSM')[0][0]]
score3 = hsIPSC.obsm['proba_RandomForest'][:,np.where(E95Classifier['Classes']=='pPSM')[0][0]]
arr = np.vstack([hsIPSC.obsm['X_draw_graph_fa'][:,0],[hsIPSC.obsm['X_draw_graph_fa'][:,1], score1, score2, score3]])
df = pd.DataFrame(np.transpose(arr), columns=['FA_1','FA_2','NeuralNet','LDA','RandomForest']) 
df.to_csv('FigS4C.csv')

# Fig4D: hsIPSC + HOX
human_hox=['HOXA1','HOXB1','HOXD1','HOXA2','HOXB2','HOXA3','HOXB3','HOXD3','HOXA4','HOXB4','HOXC4','HOXD4','HOXA5','HOXB5','HOXC5','HOXA6','HOXB6','HOXC6','HOXA7','HOXB7','HOXB8','HOXC8','HOXD8','HOXA9','HOXB9','HOXC9','HOXD9','HOXA10','HOXC10','HOXD10','HOXA11','HOXC11','HOXD11','HOXC12','HOXD12','HOXA13','HOXB13','HOXC13','HOXD13'] 
arr = hsIPSC[:,human_hox].X
df = pd.DataFrame((arr), columns=human_hox, index=hsIPSC.obs['time_id']) 
df.to_csv('FigS4D.csv')